In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
#Creating empty data drame
df= pd.DataFrame({'Name':[''], 'Speciality':[''], 'Degree':[''], 'Year_of_experience':[''] , 'Location':[''] , 'City':[''] ,'dp_score':[''], 'npv':[''], 'consultation_fee':['']})
#Creating list of cities from where data is to be extracted
lis=['Bangalore','Delhi','Mumbai']
#speciality of doctors whose data to be extracted
#Speciality = ['Dietitian/Nutritionist']
Speciality = ['Cardiologist', 'Chiropractor', 'Dentist', 'Dermatologist', 'Dietitian/Nutritionist', 'Gastroenterologist', 'bariatric surgeon', 'Gynecologist', 'Infertility Specialist', 'Neurologist', 'Neurosurgeon', 'Ophthalmologist', 'Orthopedist', 'Pediatrician', 'Physiotherapist', 'Psychiatrist', 'Pulmonologist', 'Rheumatologist', 'Urologist']
#for loop to take city name form lis and each time change url according to city
for i in lis:
    for j in Speciality:
        driver = webdriver.Chrome()
        url = f"https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22"+str(j)+"%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city="+str(i)
        driver.get(url)
        time.sleep(2)
        
        scroll_pause_time = 2 # You can set your own pause time. dont slow too slow that might not able to load more data
        screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
        A = 1

        while True:
            # scroll one screen height each time
            driver.execute_script("window.scrollTo(0, {screen_height}*{A});".format(screen_height=screen_height, A=A))
            A += 1
            time.sleep(scroll_pause_time)
            # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            # Break the loop when the height we need to scroll to is larger than the total scroll height
            if (screen_height) * A > scroll_height:
                break
        #after scrolling getting the HTML of webpage and extrancting data
        soup = BeautifulSoup(driver.page_source,'lxml')
        postings = soup.find_all('div' , class_= 'u-border-general--bottom')
        for post in postings:
            try:
                link = post.find('div' , class_ = 'listing-doctor-card').find('a').get('href')
                link_full = 'https://www.practo.com'+link
                driver.get(link_full)
                soup2 = BeautifulSoup(driver.page_source,'lxml')

                #extracting name
                try:
                    name = soup2.find('h1' , class_ = 'c-profile__title u-bold u-d-inlineblock').text
                except:
                    pass       
                #extracting degree
                try:
                    Degree = soup2.find('p' , class_ = 'c-profile__details').text
                except:
                    pass
                #extracting years of experience
                try:
                    Year_of_experience = soup2.find('div' , class_ = 'c-profile__details').find_all('h2')[-1].text
                except:
                    pass
                #extracting location
                try:
                    Location = soup2.find('h4' , class_ = 'c-profile--clinic__location').text
                except:
                    pass
                #extracting dp score
                try:
                    dp_score = soup2.find('span' , class_ = 'u-green-text u-bold u-large-font').text.strip()
                except:
                    pass
                #extracting npv
                try:
                    npv = soup2.find('span' , class_ = 'u-smallest-font u-grey_3-text').text
                except:
                    pass
                #extracting consulting fee
                try:
                    consultant_fee = soup2.find('span' , class_ = 'u-strike').text.strip()
                except:
                    consultant_fee = soup2.find('div' , class_ = 'u-f-right u-large-font u-bold u-valign--middle u-lheight-normal').text.strip()
                #appending all data into dataframe
                df = df.append({'Name':name, 'Speciality':j, 'Degree':Degree, 'Year_of_experience':Year_of_experience , 'Location':Location , 'City':i ,'dp_score':dp_score, 'npv':npv, 'consultation_fee':consultant_fee} , ignore_index = True)
            except:
                pass

In [3]:
df

,Name,Speciality,Degree,Year_of_experience,Location,City,dp_score,npv,consultation_fee
0,,,,,,,,,
1,Ms. Silky Mahajan,Dietitian/Nutritionist,"B.Sc. - Home Science (Food & Nutrition), M.Sc....",15 Years Experience Overall,"Koramangala 3 Block, Bangalore",Bangalore,97%,(270 votes),₹ 1770 fee
2,Ms. Nafeesa Imteyaz,Dietitian/Nutritionist,MSc. - Dietetics & Food Service Management,28 Years Experience Overall (27 years as spec...,"Indiranagar, Bangalore",Bangalore,98%,(3683 votes),₹ 1000 fee
3,Dr. Akhila Joshi,Dietitian/Nutritionist,"BNYS, MSc - Clinical Nutrition and Dietetics",14 Years Experience Overall (8 years as speci...,"Kaggadasapura, Bangalore",Bangalore,96%,(504 votes),₹ 500 fee
4,Ms. Vani Krishna,Dietitian/Nutritionist,MSc - Clinical Nutrition and Dietetics,11 Years Experience Overall (10 years as spec...,"Whitefield, Bangalore",Bangalore,100%,(17 votes),₹ 500 fee
...,...,...,...,...,...,...,...,...,...
129,Ms. Radhika Gawade,Dietitian/Nutritionist,BSc. - Home Science (Dietetics),20 Years Experience Overall (17 years as spec...,"Bandra, Mumbai",Mumbai,98%,(111 votes),₹ 0 fee
130,Ms. Sonal Bhosale,Dietitian/Nutritionist,B.Sc. - Home Science (Food & Nutrition),16 Years Experience Overall (10 years as spec...,"Kandivali West, Mumbai",Mumbai,98%,(111 votes),₹ 0 fee
131,Ms. Rokayya Ara,Dietitian/Nutritionist,BSc - Dietitics / Nutrition,11 Years Experience Overall (9 years as speci...,"Andheri West, Mumbai",Mumbai,98%,(111 votes),₹ 0 fee
132,Dr. Ritu Jain,Dietitian/Nutritionist,"BNYS, BSc - Dietitics / Nutrition, ND/NMD(Doct...",22 Years Experience Overall,"Andheri West, Mumbai",Mumbai,98%,(111 votes),₹ 1000 fee


In [4]:
df.to_csv('e:\Doctor_s fee prediction\doctors-fee-prediction-main\python_files\practodietitian_new5pm.csv',index=False)